In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
#pd.set_option("display.max_rows", 500)

#converts a pandas series from string to number
def to_number(series):
    series = series.str.replace(',', '') #remove thousands seperator
    series = series.str.replace('-', '0') # - dash denotes empty values
    return pd.to_numeric(series)

to_numeric = to_number #function alias

# Group S: Crime Data Report

## Loading The Data

The Crime data is supplied by the NSW Bureau of Crime Statistics and Research (BOSCAR). The data contains major crime categories, subcategories and raw crime counts per month from 1995 to 2019. 

In [2]:
crime_raw = pd.read_csv('data/CrimeLGA.gzip', compression="gzip")
crime_raw.head(3)

,LGA,Offence category,Subcategory,Jan 1995,Feb 1995,Mar 1995,Apr 1995,May 1995,Jun 1995,Jul 1995,...,Mar 2019,Apr 2019,May 2019,Jun 2019,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19
0,Albury,Homicide,Murder *,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albury,Homicide,Attempted murder,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Albury,Homicide,"Murder accessory, conspiracy",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2016 Statistics

We only use 2016 crime data for predictive modelling as it will be merged with ABS demograhic data for that year. ABS 2016 data contains the most complete information from the 2016 census. 

In [3]:
year = '2016'
month_cols = crime_raw.columns[crime_raw.columns.str.contains(year)]
columns = ['LGA', 'Offence category', 'Subcategory'] + [ c for c in month_cols]
crime = crime_raw.loc[:, columns]
crime.head(3)

,LGA,Offence category,Subcategory,Jan 2016,Feb 2016,Mar 2016,Apr 2016,May 2016,Jun 2016,Jul 2016,Aug 2016,Sep 2016,Oct 2016,Nov 2016,Dec 2016
0,Albury,Homicide,Murder *,0,0,0,0,0,0,0,0,1,0,1,0
1,Albury,Homicide,Attempted murder,0,0,0,0,0,0,0,0,0,0,1,0
2,Albury,Homicide,"Murder accessory, conspiracy",0,0,0,0,0,0,0,0,0,0,0,0


## Crime by LGA, Offence Category

Yearly totals are created for the year for each offence category within each LGA. 

In [4]:

crime['Total'] = crime[month_cols].sum(axis=1)
crime_totals= crime.drop(columns=month_cols)

crime_by_category = crime_totals.groupby(['LGA', 'Offence category']).sum()
crime_by_category = crime_by_lga = crime_by_category.reset_index()
crime_by_category.head()


,LGA,Offence category,Total
0,Albury,Abduction and kidnapping,2
1,Albury,Against justice procedures,853
2,Albury,Arson,33
3,Albury,Assault,605
4,Albury,Betting and gaming offences,1


A seperate data set it created for totals by subcategory. We could have just created one data set by subcategory and derived the category data set by grouping, but they are kept seperate for backwards compatability. 

In [5]:
crime_by_subcategory = crime_totals.groupby(['LGA', 'Offence category', 'Subcategory']).sum()
crime_by_subcategory = crime_by_lga = crime_by_subcategory.reset_index()
crime_by_subcategory.head()

,LGA,Offence category,Subcategory,Total
0,Albury,Against justice procedures,Breach Apprehended Violence Order,189
1,Albury,Against justice procedures,Breach bail conditions,526
2,Albury,Against justice procedures,Escape custody,1
3,Albury,Against justice procedures,Fail to appear,29
4,Albury,Against justice procedures,Other offences against justice procedures,22


## Offence Categories

A peek at the main categories of offences shows the most common are theft, transport, procedural, malicious damage and drug offences. Not surprisingly, the more dramatic offences such as homocide and robber are uncommon. 

In [6]:
crime_by_category.groupby(['Offence category']).sum().sort_values('Total', ascending=False)


,Total
Offence category,
Theft,234449
Transport regulatory offences,120031
Against justice procedures,65823
Assault,63245
Malicious damage to property,62822
Drug offences,47188
"Intimidation, stalking and harassment",30632
Disorderly conduct,21834
Other offences,15270


### Selecting offences

We leave out 'regulatory offences' such as transport offences, betting and gaming offences, against justice procedures. We also exclude minor/non-specific offences such as 'Disorderly conduct', 'Liquor Offences', 'Other offences'. We also leave out offences that do not have a substantial sample size such as prostitution, blackmail and extortion, homocide and pornography offences. 

Our focus will be on the following

- Malicious Damage
- Drug Offences
- Assault
- Intimidation, stalking and harassment
- Theft
- Sexual offences
- Robbery
- Arson 

Our analysis will be carried out on each of the above offences separately. This is because the nature of different types of offences may have different causes or explanations, for example - median age may be a better predictor for drug offences than theft. Analysing offences seperately may also help reduce compounding variances in the data. 



In [7]:
offence_filter = ['Malicious damage to property', 'Drug offences', 'Assault', 'Theft', 'Robbery', 'Arson', 'Intimidation, stalking and harassment', 'Sexual offences']
crime_by_category = crime_by_category[crime_by_category['Offence category'].isin(offence_filter)]
crime_by_subcategory = crime_by_subcategory[crime_by_subcategory['Offence category'].isin(offence_filter)]

## ABS population data

We join crime data from BOSCAR and ABS population data for 2016 using LGA (local government area name). Some transformation was necessary on LGA field in ABS datasets to match BOSCAR's LGA field. 

In [18]:
population_year = 2016
nsw = pd.read_csv('data/abs-population.gzip', compression="gzip")
nsw = nsw[(nsw.Code > 999) & (nsw.Code < 19000) & (nsw.Year >= 2013)] #NSw state only; NSW code '1' followed by 4 digit LGA code
nsw.Label = nsw.Label.str.replace("\(NSW\)", "") #remove state suffix
nsw.Label = nsw.Label.str.replace("\(C\)", "") #remove city suffix
nsw.Label = nsw.Label.str.replace("\(A\)", "") #remove area suffix
nsw.Label = nsw.Label.str.strip() #remove ending spaces
ppl = nsw

#create a new population dataframe
pop = pd.DataFrame()
pop['LGA'] = ppl.Label
pop['year'] = ppl.Year
#median ages
pop['age_median'] = to_number(ppl['Median Age - Persons (years)'])
pop['age_median_female'] = to_number(ppl['Median Age - Females (years)'])
pop['age_median_male'] = to_number(ppl['Median Age - Males (years)'])

#total population and population density
pop['population'] = to_number(ppl['Persons - Total (no.)'])

pop['density'] = to_number(ppl['Population density (persons/km2)'])

pop['code'] = ppl.Code #include the LGA, easier to join with other ABS datasets


pop['fertility_rate'] = to_number(ppl['Births and Deaths - Total fertility rate (per female) (rate)'])
pop['death_rate'] = to_number(ppl['Births and Deaths - Standardised death rate (per 1000 population) (rate)'])

#age bracket - children, youth, young-adult, adult, senior %
pop['children%'] =  to_number(ppl['Persons - 5-9 years (%)']) + \
                    to_number(ppl['Persons - 10-14 years (%)'])

pop['youth%'] = to_number(ppl['Persons - 15-19 years (%)']) + \
                to_number(ppl['Persons - 20-24 years (%)'])

pop['young-adult%'] =  to_number(ppl['Persons - 25-29 years (%)']) + \
                        to_number(ppl['Persons - 30-34 years (%)']) + \
    to_number(ppl['Persons - 35-39 years (%)'])

pop['adult%'] = to_number(ppl['Persons - 40-44 years (%)']) + \
                to_number(ppl['Persons - 45-49 years (%)']) + \
                to_number(ppl['Persons - 50-54 years (%)'])

pop['senior%'] = to_number(ppl['Persons - 55-59 years (%)']) + \
                to_number(ppl['Persons - 60-64 years (%)']) + \
                to_number(ppl['Persons - 65-69 years (%)']) + \
                to_number(ppl['Persons - 70-74 years (%)']) + \
                to_number(ppl['Persons - 75-79 years (%)']) + \
                to_number(ppl['Persons - 80-84 years (%)']) + \
                to_number(ppl['Persons - 85 and over (%)'])

pop['otherEnglish%'] = to_number(ppl['Speaks a Language Other Than English at Home (%)'])
pop['citizen'] = ppl['Australian citizen (%)']

#born overeas geographical regions %
pop['oceania'] = to_number(ppl['Born in Oceania and Antarctica (excluding Australia) (%)']) + \
                to_number(ppl['Aboriginal and Torres Strait Islander Peoples (%)'])
pop['overseas_born'] = to_number(ppl['Total born overseas (%)'])
pop['europe'] = to_number(ppl['Born in North-West Europe (%)']) + \
                to_number(ppl['Born in Southern and Eastern Europe (%)'])
pop['asia'] = to_number(ppl['Born in North-East Asia (%)']) + \
                to_number(ppl['Born in South-East Asia (%)']) + \
                to_number(ppl['Born in Southern and Central Asia (%)'])
pop['africa'] = to_number(ppl['Born in Sub-Saharan Africa (%)'])
pop['middle-east'] = to_number(ppl['Born in North Africa and the Middle East (%)'])
pop['americas'] = to_number(ppl['Born in Americas (%)'])


# find the unmatched LGA's
# unmatched = crime2019_merged[crime2019_merged.Code.isna()]
# unmatched.LGA.unique()
# fix name match for Gundagai/Cootamundra
# Lord Howe Island not in ABS dataset
pop.loc[pop.LGA == 'Cootamundra-Gundagai Regional', 'LGA'] = 'Cootamundra-Gundagai'
nswTS = pop
pop = pop[pop.year == population_year]

We only select a subset of features from the ABS population data, such as age bracket and ethnicity percentages. 
The ABS figures were cleaned, converted to numeric values and joined with the 2016 crime data. 

We also created a 'offence_rate' field, that being the number of offences per 10,000 people (a reasonable size for an average suburb). 

In [19]:
#PERFORM JOIN CRIME AND POPULATION DATA
crimeDS = crime_by_category.merge(pop, how="inner", left_on="LGA", right_on="LGA")
crimeSubDS = crime_by_subcategory.merge(pop, how="inner", left_on="LGA", right_on="LGA")

# create offence rate field
crimeSubDS['offence_rate'] = crimeSubDS['Total'].div(crimeSubDS['population']).mul(10000)
crimeDS['offence_rate'] = crimeDS['Total'].div(crimeDS['population']).mul(10000)

crimeDS.head(3)

,LGA,Offence category,Total,year,age_median,age_median_female,age_median_male,population,density,code,...,otherEnglish%,citizen,oceania,overseas_born,europe,asia,africa,middle-east,americas,offence_rate
0,Albury,Arson,33,2016,38.3,39.4,37.1,52171,170.5,10050,...,6.7,88.2,4.0,10.8,4.6,4.0,0.5,0.2,0.4,6.325353
1,Albury,Assault,605,2016,38.3,39.4,37.1,52171,170.5,10050,...,6.7,88.2,4.0,10.8,4.6,4.0,0.5,0.2,0.4,115.964808
2,Albury,Drug offences,645,2016,38.3,39.4,37.1,52171,170.5,10050,...,6.7,88.2,4.0,10.8,4.6,4.0,0.5,0.2,0.4,123.631903


## Income

Other detailed data sets were provided by ABS as a result of the 2016 census, such as income per LGA. 
We join this data to our crime dataset by utilizing the LGA code for the join. We only use median income 
from this dataset for our modelling

In [20]:
incomeYear = 2016
incomeABS = pd.read_csv('data/abs-income.gzip', compression="gzip")
incomeABS = incomeABS[incomeABS.Year == incomeYear]

income = pd.DataFrame()
income['code'] = incomeABS.Code
income['median_income'] = to_number(incomeABS['Median employee income ($)'])
income['mean_income'] = to_number(incomeABS['Mean employee income ($)'])

crimeDS = crimeDS.merge(income, how="left", left_on="code", right_on="code")
crimeSubDS = crimeSubDS.merge(income, how="left", left_on="code", right_on="code")
incomeABS.head(3)

,Code,Label,Year,Employee income earners (no.),Employee income earners - median age (years),Total employee income ($m),Median employee income ($),Mean employee income ($),Employee income as main source of income (%),Own unincorporated business income earners (no.),...,Commonwealth Rent Assistance (no.),Persons earning $1-$499 per week (%),Persons earning $500-$999 per week (%),Persons earning $1000-$1999 per week (%),Persons earning $2000-$2999 per week (%),Persons earning $3000 or more per week (%),Persons earning nil income (%),Persons with a negative income (%),Income inadequately described or not stated (%),Median equivalised total household income (weekly) ($)
4,10050,Albury (C),2016,"25,821",39,"1,313.2","45,382","50,857",76.8,"3,985",...,-,30.2,27.1,21.3,3.3,1.6,6.0,0.4,10.0,776
11,10130,Armidale Regional (A),2016,"13,844",39,656.0,"39,733","47,386",74.1,"3,147",...,-,32.4,25.2,17.9,3.7,1.6,8.3,0.6,10.3,744
18,10250,Ballina (A),2016,"19,767",43,951.0,"40,397","48,108",66.4,"5,158",...,-,32.2,27.8,18.6,3.2,2.1,5.5,0.3,10.2,745


## Job Sector

Job sector information may provide insights to crime rates within each LGA. We create composite fields based on percentage of the population that work in different fields in the following way:

- office workers = Clerical and administrative workers (%) + Sales workers (%)
- professionals = Managers (%) + Professionals (%) + Community and personal service workers (%)
- trades = Technicians and trades workers (%) + Machinery operators and drivers (%) + Labourers (%)

In [21]:
employ_year = 2016
employ = pd.read_csv('data/abs-employment.gzip', compression="gzip")
employ = employ[employ.Year == employ_year]
employ.head(3)

jobSectors = pd.DataFrame()
jobSectors['code'] = employ.Code

jobSectors['office'] = to_numeric(employ['Clerical and administrative workers (%)']) + to_numeric(employ['Sales workers (%)'])
jobSectors['professionals'] = to_numeric(employ['Managers (%)']) + to_numeric(employ['Professionals (%)']) + to_numeric(employ['Community and personal service workers (%)'])
jobSectors['trades'] = to_numeric(employ['Technicians and trades workers (%)']) + to_numeric(employ['Machinery operators and drivers (%)']) + to_numeric(employ['Labourers (%)'])

crimeDS = crimeDS.merge(jobSectors, how="left", left_on="code", right_on="code")
crimeSubDS  = crimeSubDS.merge(jobSectors, how="left", left_on="code", right_on="code")
jobSectors.head(3)

,code,office,professionals,trades
4,10050,24.0,42.9,31.6
11,10130,22.1,49.5,27.0
18,10250,22.2,47.4,28.7


### Local Government Area Structure

https://www.abs.gov.au/ausstats/abs@.nsf/Lookup/by%20Subject/1270.0.55.003~July%202016~Main%20Features~Local%20Government%20Areas%20(LGA)~7

## Time Series

To get some insight into overall crime trends, time series data is created from the BOSCAR dataset with totals for each year. No other covariates are merged with this dataset. 

The plots below shows the total offences for assaults and drug offences in Sydney LGA. 

In [22]:
years = range(1995, 2019, 1)
crimeTS = pd.DataFrame(columns=["LGA", "Offence category", "year", "total"])
crimeSubTS = pd.DataFrame(columns=["LGA", "Offence category", "Subcategory", "year", "total"])
crime_raw.head()
for year in years:  
    yearDF = pd.DataFrame()
    month_cols = crime_raw.columns[crime_raw.columns.str.contains(str(year))]
    columns = ['LGA', 'Offence category'] + [ c for c in month_cols]
    yearDF = crime_raw.loc[crime_raw['Offence category'].isin(offence_filter), columns] \
        .groupby(['LGA', 'Offence category']).sum().sum(axis=1).reset_index().rename(columns={0: 'total'})
    yearDF['year'] = year
    crimeTS = crimeTS.append(yearDF)
    #subcategory
    subcolumns = ['LGA', 'Offence category', 'Subcategory'] + [ c for c in month_cols]
    yearSubDF = crime_raw.loc[crime_raw['Offence category'].isin(offence_filter), subcolumns] \
        .groupby(['LGA', 'Offence category', 'Subcategory']).sum().sum(axis=1).reset_index().rename(columns={0: 'total'})
    yearSubDF['year'] = year
    crimeSubTS = crimeSubTS.append(yearSubDF)
    
crimeTS['total'] = crimeTS['total'].apply(int) #convert object to numpy.int
crimeSubTS['total'] = crimeSubTS['total'].apply(int) #convert object to numpy.int

crimeTS = crimeTS.merge(nswTS, left_on=['LGA', 'year'], right_on=['LGA', 'year'])
crimeSubTS = crimeSubTS.merge(nswTS, left_on=['LGA', 'year'], right_on=['LGA', 'year'])


## Merging Latitude and Longitude Data

A free csv containing suburb postcodes with latitude and longitudes was found at https://www.matthewproctor.com/australian_postcodes. Because the suburbs didn't map exactly to broader local government areas (LGA's) provided in the crime data, in many cases a mapping from suburb to region was applied to obtain latitudes and longitudes to regions. 

In [23]:
pc = pd.read_csv('data/postcodes.gzip', compression="gzip")
nsw = pc.loc[(pc.state == 'NSW') & (pc.type == 'Delivery Area') & (pc.id != 4327)  & (pc.id != 3300) & (pc.id != 4881), ['locality', 'lat', 'long']]
nsw.locality = nsw.locality.str.title()
replacements = {'Armidale': 'Armidale Regional', 'Bathurst': 'Bathurst Regional', 'Raymond Terrace': 'Port Stephens', 
               'Bega': 'Bega Valley', 'Katoomba': 'Blue Mountains', 'Byron Bay': 'Byron', 'Boree': 'Cabonne', 
               'Gosford': 'Central Coast', 'Wilcannia': 'Central Darling', 'Grafton': 'Clarence Valley', 
                'Gundagai': 'Cootamundra-Gundagai',
               'Auburn': 'Cumberland', 'Dubbo': 'Dubbo Regional', 'Deniliquin': 'Edward River', 
                'Hurstville': 'Georges River', 
                'Glen Innes': 'Glen Innes Severn', 'Goulburn': 'Goulburn Mulwaree', 'Holbrook': 'Greater Hume Shire',
                'Warialda': 'Gwydir', 'Bilpin': 'Hawkesbury', 'Young': 'Hilltops', 'Marrickville': 'Inner West', 
                'Gordon': 'Ku-ring-gai', 'Condobolin': 'Lachlan', 'Speers Point': 'Lake Macquarie', 
                'Quirindi': 'Liverpool Plains', 'Taree': 'Mid-Coast', 'Mudgee': 'Mid-Western Regional', 
                'Moree': 'Moree Plains', 'Moama': 'Murray River', 'Coleambally': 'Murrumbidgee', 
                'Nambucca Heads': 'Nambucca', 'Dee Why': 'Northern Beaches', 'Port Macquarie': 'Port Macquarie-Hastings',
                'Queanbeyan': 'Queanbeyan-Palerang Regional', 'Richmond': 'Richmond Valley', 'Nowra': 'Shoalhaven',
                'Cooma': 'Snowy Monaro Regional', 'Batlow': 'Snowy Valleys', 'Sutherland': 'Sutherland Shire',
                'Tamworth': 'Tamworth Regional', 'Castle Hill': 'The Hills Shire', 'Tweed Heads': 'Tweed', 
                'Scone': 'Upper Hunter Shire', 'Gunning': 'Upper Lachlan Shire', 'Coonabarabran': 'Warrumbungle Shire',
                'Grenfell': 'Weddin', 'Bowral': 'Wingecarribee', 'Picton': 'Wollondilly', 'Yass': 'Yass Valley',
                'Bankstown': 'Canterbury-Bankstown', 'Botany': 'Bayside', 'Corowa': 'Federation'
               }

for suburb, region in replacements.items():
    nsw.loc[nsw.locality == suburb, 'locality'] = region
    
crimeDS = crimeDS.merge(nsw, how='left', left_on='LGA', right_on='locality').drop(columns=['locality'])
crimeSubDS = crimeSubDS.merge(nsw, how='left', left_on='LGA', right_on='locality').drop(columns=['locality'])

In [24]:
#write dataframes as compressed files
crimeTS.to_csv("data/crimeTS.gzip", compression="gzip", index=False)
crimeSubTS.to_csv("data/crimeSubTS.gzip", compression="gzip", index=False)
crimeDS.to_csv("data/crimeDS.gzip", compression="gzip", index=False)
crimeSubDS.to_csv("data/crimeSubDS.gzip", compression="gzip", index=False)